In [ ]:
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import spacy
import gensim
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
import time
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import sys
import csv
import matplotlib.pyplot as plt
import pandas as pd
import mpu

np.set_printoptions(threshold= sys.maxsize)
np.random.seed(5)

tweet_file_name = 'tweets.xlsx'

subtitle_dic = {}
def read_csv(csv_name):
	with open(csv_name, newline='', encoding="utf8") as f:
		reader = csv.reader(f)
		tmp = list(reader)
	return tmp

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def lemmatization(doc, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    doc = nlp(doc)
    texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

stop_words = set(stopwords.words('english'))
stop_words.add('english translation')
stop_words.add('@')
def text_processing(raw_tweet):
	word_tokens = word_tokenize(raw_tweet)
	script = [w for w in word_tokens if not w in stop_words]
	script = TreebankWordDetokenizer().detokenize(script)
	script = list(gensim.utils.simple_preprocess(str(script), deacc=False))
	script = " ".join(script)
	script = lemmatization(script)
	return script

tweets = pd.read_excel(tweet_file_name)
user_tweets_dic = {}
for index, row in tweets.iterrows():
	user_tmp = row['username']
	tweets_tmp = row['tweets']
	if (tweets_tmp.upper() != tweets_tmp.lower()):
		if (user_tmp not in user_tweets_dic):
			user_tweets_dic[user_tmp] = text_processing(tweets_tmp)
		else:
			user_tweets_dic[user_tmp] = user_tweets_dic[user_tmp] + text_processing(tweets_tmp)



# for key in user_tweets_dic.keys():
# 	print(key, len(user_tweets_dic[key]))

mpu.io.write('user_tweets_dic.pickle', user_tweets_dic)
